In [ ]:
# 下記ライブラリがインストールされていなければインストールします。
!pip install opencv-python
!pip install tensorflow
!pip install tqdm

In [1]:
# ライブラリインポート
import subprocess
import csv
import numpy as np
import cv2
import tensorflow as tf
from tqdm import tqdm

ffmpegコマンドも使用します。

# 動画切り出し設定

In [27]:
# 切り出し元動画パス
src_movie = 'test.mp4'
# 切り出し秒数
cut_duration = 8
# デバッグ出力フラグ
debug = False

In [28]:
# デバッグ出力するときは、0,1ディレクトリを作る
!mkdir -p 0
!mkdir -p 1

# 動画切り出し処理

In [29]:
# TensorFlow lite の初期化
interpreter = tf.lite.Interpreter(model_path='model/model.tflite')
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [30]:
# 切り出し終了時間からこの秒数は切り出し開始しない
death_duration = 8
# 書き出しCSVファイル
with open('cut_time.csv', 'w') as f:
    writer = csv.writer(f)
    # 動画を読み込む
    cap = cv2.VideoCapture(src_movie)
    # フレーム数を取得
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    # 1秒あたりフレーム数を取得
    fps = cap.get(cv2.CAP_PROP_FPS)
    # 0.5秒に1回予測する
    skip = fps / 2
    # フレーム
    i = 0
    # 切り出し開始しないカウントダウン
    no_start = 0
    # デバッグ出力画像ファイルインデックス
    out_index = 0
    for i in tqdm(range(frame_count)):
        ret, img = cap.read()
        if ret:
            if i % skip == 0 and no_start == 0:
                # フレームを予測する大きさに縮小
                shrink = cv2.resize(
                    img, (224, 224), interpolation=cv2.INTER_CUBIC)
                # 4次元に変換する
                input_tensor = shrink.reshape(1, 224, 224, 3)
                # それをTensorFlow liteに指定する
                interpreter.set_tensor(input_details[0]['index'], input_tensor)
                # 推論実行
                interpreter.invoke()
                # 出力層を確認
                output_tensor = interpreter.get_tensor(output_details[0]['index'])
                # やられたシーン判定
                scene = np.argmax(output_tensor)
                if scene == 1:
                    # やられたシーンの時は
                    # 切り出し開始秒数を出力
                    ss = i - cut_duration * fps
                    if ss < 0:
                        ss = 0
                    writer.writerow(["%d.%02d" % (ss/fps, 100 * (ss % fps)/fps)])
                    # シーン判定をしばらく止める
                    no_start = fps * death_duration
                # デバッグ出力
                if debug:
                    out_path = "%d/frame%05d.jpg" % (scene, out_index)
                    cv2.imwrite(out_path, img)
                    out_index += 1
            if no_start >= 1:
                no_start -= 1
        else:
            break

100%|██████████| 431717/431717 [33:59<00:00, 211.66it/s]


In [24]:
# やられたシーン切り出し動画の出力ディレクトリ
!mkdir -p extract

In [25]:
# CSVファイルから切り出し開始時刻配列を作成する
sss = []
with open('cut_time.csv') as f:
    reader = csv.reader(f)
    for row in reader:
        sss.append(row[0])

In [26]:
# ffmpegで切り出す
for i in tqdm(range(len(sss))):
    ss = sss[i]
    command = "ffmpeg -y -ss %s -i %s -t %d -c copy extract/scene%03d.mp4" % (ss, src_movie, cut_duration, i)
    subprocess.run(command, shell=True)

100%|██████████| 49/49 [00:03<00:00, 16.30it/s]
